In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# 🌊 Core Libraries
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 🧠 TensorFlow & Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import (
    ImageDataGenerator,
    load_img,
    img_to_array,
)

# 🧱 Keras Layers
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    BatchNormalization,
    MaxPool2D,
    Dropout,
    Dense,
    GlobalAveragePooling2D,
    Flatten,
    Add,
)

seed_value = 42

np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# 📂 Explore dataset paths
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!ls ./kaggle/working

ships32


## Exploration, Analyse, Data Preprocessing

In [ ]:
new_train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.9, 1.1],
    fill_mode="nearest",
    validation_split=0.2,
)

data = "/kaggle/working/ships32"
batch_size = 128

train_generator = new_train_datagen.flow_from_directory(
    data,
    target_size=(32, 32),
    batch_size=batch_size,
    class_mode="sparse",
    subset="training",
    shuffle=True,
    seed=42,
)

val_generator = new_train_datagen.flow_from_directory(
    data,
    target_size=(32, 32),
    batch_size=batch_size,
    class_mode="sparse",
    subset="validation",
    shuffle=False,
    seed=42,
)


## Model

In [ ]:
def BoatNet_30(input_shape=(32, 32, 3), num_classes=13):
    inputs = Input(shape=input_shape)

    # Block 1
    x = Conv2D(
        64, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(2e-4)
    )(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(
        64, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(2e-4)
    )(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Block 2
    x = Conv2D(
        128, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(2e-4)
    )(x)
    x = BatchNormalization()(x)
    x = Conv2D(
        128, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(2e-4)
    )(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    # Residual Block
    # Shortcut before main path
    shortcut = Conv2D(128, (1, 1), padding="same")(x)  # 1
    # Main path
    res = Conv2D(128, (3, 3), padding="same", activation="relu")(x)  # 2
    res = BatchNormalization()(res)  # 3
    # Add skip connection
    x = Add()([shortcut, res])

    # Block 3
    x = Conv2D(
        256, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(2e-4)
    )(x)
    x = BatchNormalization()(x)
    x = Conv2D(
        256, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(2e-4)
    )(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.4)(x)

    # Block 4
    x = Conv2D(
        512, (3, 3), padding="same", activation="relu", kernel_regularizer=l2(2e-4)
    )(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    # Classification
    x = GlobalAveragePooling2D()(x)
    x = Dense(768, activation="relu", kernel_regularizer=l2(2e-4))(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes)(x)  # from_logits=True for sparse loss

    return Model(inputs=inputs, outputs=outputs)


In [ ]:
model = BoatNet_30()

initial_learning_rate = 1e-3
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=5000, decay_rate=0.9, staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


model.summary()
print(f"Number of layers: {len(model.layers)}")

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=5, restore_best_weights=True, verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        "best_ship_model.keras", save_best_only=True, monitor="val_accuracy", verbose=1
    ),
]

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=callbacks,
    verbose=1,
)


## Résultat à soumettre

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"], loc="lower right")

plt.subplot(1, 2, 2)
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"], loc="upper right")
plt.tight_layout()
plt.savefig("training_history.png")
plt.show()

In [ ]:
val_loss, val_acc = model.evaluate(val_generator)
print(f"Final validation accuracy: {val_acc:.4f}")
print(f"Final validation loss: {val_loss:.4f}")

In [ ]:
model.save("final_ship_model.keras")

In [3]:
X_test = np.load("./kaggle/input/navires-2025/ships_competition.npz", allow_pickle=True)[
    "X"
]
X_test = X_test.astype("float32") / 255


In [5]:
res = model.predict(X_test).argmax(axis=1)
df = pd.DataFrame({"Category": res})
df.to_csv("reco_nav.csv", index_label="Id")



63/63 ━━━━━━━━━━━━━━━━━━━━ 26s 409ms/step


In [5]:
!head reco_nav.csv


Id,Category
0,5
1,3
2,8
3,6
4,5
5,0
6,1
7,1
8,3


In [6]:
import os

os.chdir(r"./kaggle/working")
from IPython.display import FileLink

FileLink(r"reco_nav.csv")


/Users/erwinrodrigues/project/Project-ConvNN/kaggle/working/reco_nav.csv